In [40]:
# first attempt at multi task learning and integration with topological quantum chemistry database
import e3nn.util
import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric
import torch_scatter

import e3nn
from e3nn import o3
#from e3nn.util.datatypes import DataPeriodicNeighbors
#from e3nn.nn._gate import GatedConvParityNetwork
#from e3nn.math._linalg import Kernel

import pymatgen as mg
import pymatgen.io
from pymatgen.core.structure import Structure
import pymatgen.analysis.magnetism.analyzer as pg
from mp_api.client import MPRester
import numpy as np
import pickle
from mendeleev import element
import matplotlib.pyplot as plt

from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

import io
import random
import math
import sys
import time, os
import datetime
from pathlib import Path
from dotenv import load_dotenv


In [41]:
load_dotenv()
load_dotenv(Path("/Users/abiralshakya/Documents/Research/Topological_Insulators_OnGithub/generative_nmti/Integrated_Magnetic_Topological/matprojectapi.env"))
api_key = os.getenv("MP_API_KEY")

In [42]:
# %% Process Materials Project Data
order_list_mp = []
structures_list_mp = []
formula_list_mp = []
sites_list = []
id_list_mp = []
y_values_mp = []
order_encode = {"NM": 0, "AFM": 1, "FM": 2, "FiM": 2}
topo_encode = {False: 0, True: 1}


mp_structures_dict = torch.load('/Users/abiralshakya/Documents/Research/Topological_Insulators_OnGithub/generative_nmti/Integrated_Magnetic_Topological/magnetic_order/preload_data/mp_structures_2025-04-07_11-39.pt', 
                                weights_only= False)
structures = mp_structures_dict['structures']

structures_copy = structures.copy()
# for struc in structures_copy:
#     print(struc)
#     # if len(struc["structur"]) > 250:
#     #     structures.remove(struc)
#     #     print("MP Structure Deleted")

print(type(structures))

# If it's a dictionary:
if isinstance(structures, dict):
    print("Keys:", list(structures.keys())[:5])
    print("Sample value type:", type(list(structures.values())[0]))

# If it's something else (like a pandas DataFrame), this will help too:
try:
    print(structures.head())
except:
    pass

<class 'list'>


In [44]:
order_list = []
for i in range(len(structures)):
    order = pg.CollinearMagneticStructureAnalyzer(structures[i])
    order_list.append(order.ordering.name)
id_NM = []
id_FM = []
id_AFM = []
for i in range(len(structures)):
    if order_list[i] == 'NM':
        id_NM.append(i)
    if order_list[i] == 'AFM':
        id_AFM.append(i)
    if order_list[i] == 'FM' or order_list[i] == 'FiM':
        id_FM.append(i)
np.random.shuffle(id_FM)
np.random.shuffle(id_NM)
np.random.shuffle(id_AFM)
id_AFM, id_AFM_to_delete = np.split(id_AFM, [int(len(id_AFM))])
id_NM, id_NM_to_delete = np.split(id_NM, [int(1.2 * len(id_AFM))])
id_FM, id_FM_to_delete = np.split(id_FM, [int(1.2 * len(id_AFM))])

structures_mp = [structures[i] for i in id_NM] + [structures[j] for j in id_FM] + [structures[k] for k in id_AFM]
np.random.shuffle(structures_mp)


In [51]:
for structure in structures_mp:
    analyzed_structure = pg.CollinearMagneticStructureAnalyzer(structure)
    order_list_mp.append(analyzed_structure.ordering)
    structures_list_mp.append(structure)
    formula_list_mp.append(structure.formula)
    id_list_mp.append(structure.material_id)
    sites_list.append(structure.nsites)

def get_topological_insulator_label(material_id):
    with MPRester(api_key = api_key) as mpr:
        try:
            data = mpr.get_data(material_id, fields=["is_topological_insulator"])
            return data[0]["is_topological_insulator"]
        except Exception as e:
            print(f"Error getting topological insulator label for {material_id}: {e}")
            return False

AttributeError: 'Structure' object has no attribute 'material_id'